# Project 06 — Volatility Modeling (GARCH Family) + Advanced Pricing (Implied Vol & Heston)

**Goal:** build a clean bridge from **returns → volatility → option pricing**.

You will:
- Download market data and compute log-returns
- Fit **GARCH(1,1)** and **GJR-GARCH(1,1)** with **Normal** and **Student-t** innovations
- Compare model fits (AIC/BIC) and visualize **conditional volatility**
- Price a European call via **Black–Scholes** using different volatility inputs
- Compute **implied volatility** (invert Black–Scholes)
- Price options under **Heston stochastic volatility** and visualize the **smile/skew**


## Environment / requirements

Recommended Python packages:
- `numpy`, `pandas`
- `plotly`, `ipywidgets`
- `yfinance`
- `scipy`
- `arch`

If you are missing a package, run (in a terminal):

```bash
pip install numpy pandas plotly ipywidgets yfinance scipy arch
```


In [23]:

from __future__ import annotations
from pathlib import Path

SEED = 42

PROJECT_DIR = Path.cwd()
ASSETS_DIR = PROJECT_DIR / "assets"
ASSETS_DIR.mkdir(parents=True, exist_ok=True)

print("CWD:", PROJECT_DIR)
print("ASSETS_DIR:", ASSETS_DIR.resolve())


CWD: c:\Users\Karim\Desktop\quant-finance-portfolio\projects\06_volatility_garch_heston_pricing
ASSETS_DIR: C:\Users\Karim\Desktop\quant-finance-portfolio\projects\06_volatility_garch_heston_pricing\assets


In [24]:
import numpy as np
import pandas as pd
import math
import plotly.express as px
import plotly.graph_objects as go
import yfinance as yf
from scipy.optimize import brentq
from scipy.stats import norm
from scipy.integrate import quad
from functools import lru_cache
# Volatility models
from arch import arch_model


## 1) Data: prices → log-returns

We use daily log-returns:

\[
r_t = \log(S_t) - \log(S_{t-1})
\]

Why log-returns?
- they are additive over time
- they match standard continuous-time models (GBM, Heston, ...)


In [25]:

def download_prices(ticker: str = "AAPL", start: str = "2015-01-01") -> pd.Series:
    s = yf.download(ticker, start=start, auto_adjust=True, progress=False)["Close"].dropna()
    s.name = "close"
    return s

def log_returns(prices: pd.Series) -> pd.Series:
    r = np.log(prices).diff().dropna()
    r.name = "r"
    return r

ticker = "AAPL"
prices = download_prices(ticker, start="2015-01-01")
r = log_returns(prices)

print(ticker, "prices:", prices.shape, "returns:", r.shape)
r.head()


AAPL prices: (2758, 1) returns: (2757, 1)


Ticker,AAPL
Date,
2015-01-05,-0.028576
2015-01-06,0.000094
2015-01-07,0.013925
2015-01-08,0.037703
2015-01-09,0.001072


In [26]:
# Defensive: recompute log-returns from the current price series so this cell works even after a kernel restart.
p = prices.copy() if "prices" in globals() else (adj_close.copy() if "adj_close" in globals() else None)
if p is None:
    raise NameError("No price series found. Run the data download cell first (prices / adj_close).")

if isinstance(p, pd.DataFrame):
    # If multiple columns, take the first numeric one
    if p.shape[1] == 1:
        p = p.iloc[:, 0]
    else:
        p = p.select_dtypes(include=[np.number]).iloc[:, 0]

p = p.dropna()
r = np.log(p).diff().dropna()
r.name = "r"

df = r.to_frame()

df["roll_vol_21d_ann"] = df["r"].rolling(21).std(ddof=1) * np.sqrt(252)

fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=df.index, y=df["r"], mode="lines", name="returns"))
fig1.update_layout(template="plotly_dark", title="Daily log-returns", xaxis_title="Date", yaxis_title="r_t")
fig1.show()
fig1.write_html(ASSETS_DIR / "returns_timeseries.html")

fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=df.index, y=df["roll_vol_21d_ann"], mode="lines", name="Rolling vol (21d, ann.)"))
fig2.update_layout(template="plotly_dark", title="Rolling volatility (annualized)", xaxis_title="Date", yaxis_title="Vol")
fig2.show()
fig2.write_html(ASSETS_DIR / "rolling_vol_21d.html")

fig3 = px.histogram(df, x="r", nbins=120, template="plotly_dark", title="Return distribution (histogram)")
fig3.show()
fig3.write_html(ASSETS_DIR / "returns_hist.html")

### How to interpret these charts
- **Returns**: look random day-to-day, but not “i.i.d.” in volatility.
- **Rolling volatility**: you see clusters (calm → turbulent → calm).
- **Histogram**: more mass in the tails than a Normal is typical (“fat tails”).


## 2) GARCH family (volatility clustering + leverage effect)

**GARCH(1,1)**:
\[
\sigma_t^2 = \omega + \alpha r_{t-1}^2 + \beta \sigma_{t-1}^2
\]

**GJR-GARCH(1,1)** (leverage effect):
\[
\sigma_t^2 = \omega + \alpha r_{t-1}^2 + \gamma r_{t-1}^2\mathbf{1}_{r_{t-1}<0} + \beta \sigma_{t-1}^2
\]

We fit both with **Normal** and **Student-t** innovations.


In [27]:

# ARCH library convention: returns in percent scale
r_pct = 100 * r

def fit_garch_family(r_pct: pd.Series):
    fits = {}

    # GARCH(1,1) Normal
    fits["GARCH-N"] = arch_model(r_pct, mean="Zero", vol="Garch", p=1, q=1, dist="normal").fit(disp="off")

    # GARCH(1,1) Student-t
    fits["GARCH-t"] = arch_model(r_pct, mean="Zero", vol="Garch", p=1, q=1, dist="t").fit(disp="off")

    # GJR-GARCH(1,1) Normal (o=1 adds the asymmetric term)
    fits["GJR-N"] = arch_model(r_pct, mean="Zero", vol="Garch", p=1, o=1, q=1, dist="normal").fit(disp="off")

    # GJR-GARCH(1,1) Student-t
    fits["GJR-t"] = arch_model(r_pct, mean="Zero", vol="Garch", p=1, o=1, q=1, dist="t").fit(disp="off")

    return fits

fits = fit_garch_family(r_pct)

model_cmp = pd.DataFrame({
    "Model": list(fits.keys()),
    "AIC": [fits[k].aic for k in fits],
    "BIC": [fits[k].bic for k in fits],
}).sort_values("AIC")

model_cmp


,Model,AIC,BIC
3,GJR-t,10276.721876,10306.331368
1,GARCH-t,10326.806127,10350.493721
2,GJR-N,10499.777250,10523.464844
0,GARCH-N,10586.803327,10604.569022


In [28]:

# Plot conditional volatility (annualized)
vol_df = pd.DataFrame(index=r.index)
for name, res in fits.items():
    # conditional_volatility is daily %; convert to annualized decimal
    vol_df[name] = (res.conditional_volatility / 100.0) * np.sqrt(252)

fig = go.Figure()
for c in vol_df.columns:
    fig.add_trace(go.Scatter(x=vol_df.index, y=vol_df[c], mode="lines", name=c))
fig.update_layout(template="plotly_dark",
                  title="Conditional volatility (annualized) — GARCH family comparison",
                  xaxis_title="Date", yaxis_title="Vol")
fig.show()
fig.write_html(ASSETS_DIR / "garch_family_vol_paths.html")


### Interpretation (what you should say)
- GARCH explains **volatility clustering** (high vol persists).
- GJR reacts more when returns are negative → **leverage effect**.
- If Student-t wins on AIC/BIC, that supports **fat tails** in residuals.


## 3) Black–Scholes pricing + implied volatility

Black–Scholes (no dividends) for a call:
\[
C = S_0\Phi(d_1) - K e^{-rT}\Phi(d_2)
\]
\[
d_1 = \frac{\ln(S_0/K) + (r+\tfrac{1}{2}\sigma^2)T}{\sigma\sqrt{T}},\quad d_2 = d_1 - \sigma\sqrt{T}
\]

**Implied volatility**: given a market price \(C_{mkt}\), solve \(C_{BS}(\sigma)=C_{mkt}\).


In [29]:
def bs_call_price(S0: float, K: float, T: float, r: float, sigma: float, q: float = 0.0) -> float:
    """Black–Scholes European call price with continuous dividend yield q."""
    if T <= 0.0:
        return max(S0 - K, 0.0)
    if sigma <= 0.0:
        forward = S0 * math.exp((r - q) * T)
        disc = math.exp(-r * T)
        return disc * max(forward - K, 0.0)

    sqrtT = math.sqrt(T)
    d1 = (math.log(S0 / K) + (r - q + 0.5 * sigma * sigma) * T) / (sigma * sqrtT)
    d2 = d1 - sigma * sqrtT
    return (S0 * math.exp(-q * T) * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2))


def implied_vol_call(
    price: float,
    S0: float,
    K: float,
    T: float,
    r: float,
    q: float = 0.0,
    lo: float = 1e-8,
    hi: float = 5.0,
) -> float:
    """Implied vol (call) via robust bracketing + Brent.

    Returns np.nan if the price is outside arbitrage bounds or if no bracket is found.
    """
    price = float(price)
    if T <= 0.0:
        return np.nan

    disc_q = math.exp(-q * T)
    disc_r = math.exp(-r * T)
    intrinsic = max(S0 * disc_q - K * disc_r, 0.0)
    upper = S0 * disc_q

    if price < intrinsic - 1e-10 or price > upper + 1e-10:
        return np.nan

    def f(sig: float) -> float:
        return bs_call_price(S0, K, T, r, sig, q=q) - price

    a = float(lo)
    b = float(hi)
    fa = f(a)
    fb = f(b)

    max_b = 20.0
    while fa * fb > 0.0 and b < max_b:
        b *= 1.5
        fb = f(b)

    if fa * fb > 0.0:
        return np.nan

    return float(brentq(f, a, b, maxiter=200))


In [30]:

S0 = float(prices.iloc[-1])
K = S0
T = 30/365
r_rf = 0.02

hist_vol = float(r.rolling(63).std(ddof=1).iloc[-1] * np.sqrt(252))  # 3 months realized
best_model = model_cmp.iloc[0]["Model"]
best_fit = fits[best_model]

# 1-step ahead variance forecast from ARCH
fcst = best_fit.forecast(horizon=1, reindex=False)
sigma1d_pct = float(np.sqrt(fcst.variance.values[-1, 0]))  # daily vol in %
sigma1d = sigma1d_pct / 100.0
garch_vol = float(sigma1d * np.sqrt(252))

C_hist = bs_call_price(S0, K, T, r_rf, hist_vol)
C_garch = bs_call_price(S0, K, T, r_rf, garch_vol)

pd.DataFrame(
    {"sigma": [hist_vol, garch_vol], "BS_call": [C_hist, C_garch]},
    index=["Realized vol (63d)", f"{best_model} forecast"]
).round(6)


C:\Users\Karim\AppData\Local\Temp\ipykernel_26528\3334164509.py:1: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead



,sigma,BS_call
Realized vol (63d),0.196211,6.33513
GJR-t forecast,0.198595,6.40930


### Interpretation
Same Black–Scholes model, different volatility inputs:
- realized vol is backward-looking,
- GARCH vol is model-based and reacts after shocks.

This is a clean way to show **why volatility modeling matters**.


## 4) Heston stochastic volatility (smile / skew)

Heston model:
\[
\begin{aligned}
dS_t &= r S_t\,dt + \sqrt{v_t} S_t\,dW_t^{(1)} \\
dv_t &= \kappa(\theta - v_t)\,dt + \xi\sqrt{v_t}\,dW_t^{(2)},\quad dW^{(1)}dW^{(2)}=\rho\,dt
\end{aligned}
\]

Intuition:
- volatility is random (\(v_t\) changes through time),
- correlation \(\rho<0\) generates **equity skew** (sell-offs → vol jumps),
- this produces an implied-vol **smile/skew** not captured by Black–Scholes.


In [31]:

def heston_charfunc(u, S0, T, r, kappa, theta, xi, rho, v0, q=0.0):
    # Characteristic function of ln(S_T) under risk-neutral measure.
    # Uses a stable "little Heston trap" style formulation.
    x0 = np.log(S0)
    a = kappa * theta
    b = kappa
    iu = 1j * u

    d = np.sqrt((rho * xi * iu - b)**2 + (xi**2) * (iu + u**2))
    g = (b - rho * xi * iu - d) / (b - rho * xi * iu + d)

    exp_dt = np.exp(-d * T)

    C = (r - q) * iu * T + (a / (xi**2)) * ((b - rho * xi * iu - d) * T - 2.0 * np.log((1 - g * exp_dt) / (1 - g)))
    D = ((b - rho * xi * iu - d) / (xi**2)) * ((1 - exp_dt) / (1 - g * exp_dt))

    return np.exp(C + D * v0 + iu * x0)

def heston_call_price(S0, K, T, r, kappa, theta, xi, rho, v0, q=0.0, u_max=100.0):
    # Semi-closed form using probabilities P1/P2 integrals
    if T <= 0:
        return max(0.0, S0 - K)

    lnK = np.log(K)

    phi_mi = heston_charfunc(-1j, S0, T, r, kappa, theta, xi, rho, v0, q=q)

    def P2_integrand(u):
        phi = heston_charfunc(u, S0, T, r, kappa, theta, xi, rho, v0, q=q)
        return np.real(np.exp(-1j * u * lnK) * phi / (1j * u))

    def P1_integrand(u):
        phi = heston_charfunc(u - 1j, S0, T, r, kappa, theta, xi, rho, v0, q=q)
        return np.real(np.exp(-1j * u * lnK) * phi / (1j * u * phi_mi))

    # Avoid u=0 singularity by starting at eps
    eps = 1e-6
    P1 = 0.5 + (1.0 / np.pi) * quad(lambda uu: P1_integrand(uu), eps, u_max, limit=200)[0]
    P2 = 0.5 + (1.0 / np.pi) * quad(lambda uu: P2_integrand(uu), eps, u_max, limit=200)[0]

    return float(S0 * np.exp(-q*T) * P1 - K * np.exp(-r*T) * P2)


In [32]:

# Quick sanity check: Heston price should be close to BS if vol-of-vol is small and v0=theta=sigma^2

S0 = float(prices.iloc[-1])
r_rf = 0.02
q_div = 0.0
T = 30/365
K = S0

sigma = float(r.rolling(63).std(ddof=1).iloc[-1] * np.sqrt(252))
v0 = sigma**2
theta = v0

# "near-BS" parameters
kappa = 2.0
xi = 0.05
rho = -0.3

C_heston = heston_call_price(S0, K, T, r_rf, kappa, theta, xi, rho, v0, q=q_div, u_max=80.0)
C_bs = bs_call_price(S0, K, T, r_rf, sigma)

pd.DataFrame({"price": [C_bs, C_heston]}, index=["Black–Scholes", "Heston (near-BS)"]).round(6)


C:\Users\Karim\AppData\Local\Temp\ipykernel_26528\1845376369.py:3: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead



,price
Black–Scholes,6.335130
Heston (near-BS),6.333946


## 5) Heston smile (interactive)

We compute Heston prices for a range of strikes, then invert Black–Scholes to get an implied-vol curve.

**Interpretation:**  
- If \(\rho<0\), implied vol tends to be higher for low strikes → equity **skew**.
- Higher \(\xi\) (vol-of-vol) increases curvature of the smile.


In [33]:

def _scalar(x) -> float:
    """Best-effort conversion to float (handles pandas Series with 1 element)."""
    try:
        arr = np.asarray(x)
        return float(arr.reshape(-1)[0])
    except Exception:
        try:
            return float(x)
        except Exception:
            return float("nan")


@lru_cache(maxsize=4096)
def _heston_call_cached(S0, K, T, r, kappa, theta, xi, rho, v0, q, u_max):
    return heston_call_price(S0, K, T, r, kappa, theta, xi, rho, v0, q=q, u_max=u_max)

def heston_smile(S0, T, r, q, params, strikes):
    kappa, theta, xi, rho, v0 = params
    prices = []
    ivs = []
    for K in strikes:
        C = _heston_call_cached(S0, float(K), float(T), float(r), float(kappa), float(theta), float(xi), float(rho), float(v0), float(q), 80.0)
        prices.append(C)
        ivs.append(implied_vol_call(C, S0, float(K), T, r, q=q))
    return np.array(prices), np.array(ivs)

S0 = float(prices.iloc[-1])
r_rf = 0.02
q_div = 0.0
T0 = 30/365

# default params
sigma0 = float(r.rolling(63).std(ddof=1).iloc[-1] * np.sqrt(252))
v0_ = sigma0**2
theta_ = v0_

strikes = np.linspace(0.7*S0, 1.3*S0, 25)

C_h, iv_h = heston_smile(S0, T0, r_rf, q_div, (2.0, _scalar(theta_), 0.40, -0.6, _scalar(v0_)), strikes)

fig = go.Figure()
fig.add_trace(go.Scatter(x=strikes, y=iv_h, mode="lines+markers", name="Heston IV"))
fig.update_layout(template="plotly_dark", title="Heston implied volatility smile (T=30d)", xaxis_title="Strike K", yaxis_title="Implied vol")
fig.show()
fig.write_html(ASSETS_DIR / "heston_iv_smile.html")


C:\Users\Karim\AppData\Local\Temp\ipykernel_26528\1194001901.py:27: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead



### Final interview pitch (one-liner)
“I modeled volatility with GARCH/GJR (Normal vs Student-t), used forecasts as vol inputs in Black–Scholes, and then showed how Heston generates skew/smile via stochastic volatility and correlation.”
